In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

169009152/169001437 [==============================] - 4s 0us/step


# Transfer Learning

In [3]:
from tensorflow import keras

base_model = keras.applications.DenseNet169(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(32, 32, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

2021-09-15 09:13:55.914166: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-15 09:13:55.917938: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-15 09:13:55.966760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 09:13:55.967443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-15 09:13:55.967499: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-15 09:13:56.001054: I tensorflow/stream_executor/platform/def

51879936/51877672 [==============================] - 1s 0us/step


In [4]:
base_model.trainable = False

In [5]:
inputs = keras.Input(shape=(32, 32, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(100)(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

%time model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

model.evaluate(test_images,  test_labels, verbose=2)

2021-09-15 09:14:04.310178: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-15 09:14:04.321266: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000140000 Hz


Epoch 1/5


2021-09-15 09:14:11.567204: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-15 09:14:12.371183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-15 09:14:12.393871: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


1563/1563 [==============================] - 61s 31ms/step - loss: 3.3276 - accuracy: 0.2336 - val_loss: 2.5930 - val_accuracy: 0.3578
Epoch 2/5
1563/1563 [==============================] - 45s 29ms/step - loss: 2.3094 - accuracy: 0.4160 - val_loss: 2.5053 - val_accuracy: 0.3744
Epoch 3/5
1563/1563 [==============================] - 45s 29ms/step - loss: 2.0855 - accuracy: 0.4642 - val_loss: 2.4874 - val_accuracy: 0.3851
Epoch 4/5
1563/1563 [==============================] - 45s 29ms/step - loss: 1.9601 - accuracy: 0.4910 - val_loss: 2.4770 - val_accuracy: 0.3860
Epoch 5/5
1563/1563 [==============================] - 45s 29ms/step - loss: 1.8511 - accuracy: 0.5148 - val_loss: 2.4979 - val_accuracy: 0.3879
CPU times: user 4min 1s, sys: 5.04 s, total: 4min 6s
Wall time: 4min 1s
313/313 - 7s - loss: 2.4979 - accuracy: 0.3879


[2.497878074645996, 0.3878999948501587]

In [7]:
model.save('model.h5')
model = keras.models.load_model('model.h5')
model.make_predict_function()

In [11]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img_height = img_width = 32
img = keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format([np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to [61] with a 96.58 percent confidence.


In [ ]:
label_train